# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [19]:
from io import BytesIO
from datetime import datetime
import csv
from pathlib import Path

import requests
import pdfplumber

In [20]:
url = "https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_9.2.2025.pdf"
req = requests.get(url)

In [21]:
pdf = pdfplumber.open(BytesIO(req.content))

In [22]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [23]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [24]:
lines

[['Statewide Report by County - September 2, 2025',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'Inactive',
  'Total Active'],
 ['Aurora', '504', '2', '1,007', '232', '3', '99', '42', '1,847'],
 ['Beadle', '2,458', '28', '5,227', '1,377', '45', '1,211', '425', '10,346'],
 ['Bennett', '666', '8', '731', '190', '2', '201', '110', '1,798'],
 ['Bon Homme', '868', '10', '2,430', '492', '', '242', '173', '4,042'],
 ['Brookings',
  '5,114',
  '107',
  '10,182',
  '3,661',
  '23',
  '2,891',
  '836',
  '21,978'],
 ['Brown', '7,089', '103', '13,034', '3,574', '15', '2,021', '487', '25,836'],
 ['Brule', '876', '12', '1,774', '461', '4', '225', '88', '3,352'],
 ['Buffalo', '533', '2', '155', '79', '4', '101', '54', '874'],
 ['Butte', '758', '55', '4,664', '874', '21', '667', '891', '7,039'],
 ['Campbell', '103', '1', '908', '73', '', '50', '44', '1,135'],
 ['Charles Mix

In [25]:
# report date is in the first line
report_date = lines.pop(0)

In [26]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2025-09-02


In [27]:
# save the PDF
outpath = Path('pdfs') / f'{report_date}.pdf'

with open(outpath, 'wb') as outfile:
    outfile.write(req.content)

print(f'Wrote {outpath}')

Wrote pdfs/2025-09-02.pdf


In [28]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [29]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [30]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'inactive',
 'total active']

In [31]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [32]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [33]:
data[0]

{'county': 'Aurora',
 'democratic': 504,
 'libertarian': 2,
 'republican': 1007,
 'independent': 232,
 'other': 3,
 'npa': 99,
 'inactive': 42,
 'date': '2025-09-02'}

In [34]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [35]:
# write to file
filepath = (Path('data') / f'{report_date}.csv').resolve()

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: /Users/cjwinchester/south-dakota-voter-registration-data/data/2025-09-02.csv


In [36]:
%run build_files.py

Party list: americans_elect, constitution, democratic, inactive, independent, libertarian, no_labels, npa, npa_ind, npa_ind_oth, other, reform, republican
--------------------
Wrote sd-voter-registration-data.csv
Wrote sd-voter-registration-data-simplified.csv
Wrote README.md
